# OpenAI Function Calling


In [4]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole, chat_completion

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

giga = GigaChat(credentials=api_key, verify_ssl_certs=False)

In [5]:
import json

# Пример фиктивной функции, жестко запрограммированной для возврата той же погоды
# В рабочей среде это может быть ваш внутренний API или внешний API
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""

    weather_info = {
        "location": location,
        "temperature": "22",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [6]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Узнать погоду в заданной локации",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Город, например Санкт-Петербург",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [118]:
messages = [
    {
        "role": "user",
        "content": "Какая погода в Boston по Фаренгейту?"
    }
]

In [7]:
def get_completition(messages: list, 
                     functions: list, 
                     function_call: str = 'auto') -> chat_completion.ChatCompletion:

    payload = Chat(
        messages=messages,
        functions=functions,
        function_call=function_call
    )

    response = giga.chat(payload)
    return response

In [120]:
response = get_completition(messages, functions)

In [121]:
print(response)

choices=[Choices(message=Messages(role='assistant', content='', function_call=FunctionCall(name='get_current_weather', arguments={'location': 'Boston', 'unit': 'fahrenheit'}), name=None, attachments=None, data_for_context=None, functions_state_id='80443227-aa68-4c9f-b197-7b13764763c8', id_=None), index=0, finish_reason='function_call')] created=1735908229 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=117, completion_tokens=42, total_tokens=159) object_='chat.completion'


In [123]:
response_message = response.choices[0].message

In [124]:
response_message.content

''

In [125]:
response_message.function_call

FunctionCall(name='get_current_weather', arguments={'location': 'Boston', 'unit': 'fahrenheit'})

In [126]:
args = response_message.function_call.arguments

In [127]:
get_current_weather(**args)

Boston
fahrenheit


'{"location": "Boston", "temperature": "22", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

* Pass a message that is not related to a function.

In [128]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [129]:
response = get_completition(messages, functions)

In [130]:
print(response)

choices=[Choices(message=Messages(role='assistant', content='Hello! How can I assist you today?', function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id='8ddcfb6a-6676-4844-a165-2346350f10ff', id_=None), index=0, finish_reason='stop')] created=1735908272 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=107, completion_tokens=14, total_tokens=121) object_='chat.completion'


* Pass additional parameters to force the model to use or not a function.

In [131]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = get_completition(messages, functions, function_call='auto')
print(response)

choices=[Choices(message=Messages(role='assistant', content='Hello! How can I assist you today?', function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id='f97b6762-9b33-4312-9e4b-c670e2454072', id_=None), index=0, finish_reason='stop')] created=1735908283 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=107, completion_tokens=14, total_tokens=121) object_='chat.completion'


* Use mode 'none' for function call.

In [132]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = get_completition(messages, functions, function_call='none')
print(response)

choices=[Choices(message=Messages(role='assistant', content='Hello! How can I assist you today?', function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id=None, id_=None), index=0, finish_reason='stop')] created=1735908260 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=13, completion_tokens=10, total_tokens=23) object_='chat.completion'


* When the message should call a function and still uses mode 'none'.

In [139]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = get_completition(messages, functions, function_call='none')
print(response)

choices=[Choices(message=Messages(role='assistant', content="I can't provide information about the current weather or forecasts as I don't have access to real-time data. However, you can check the weather for Boston using a weather app, website, or by asking a virtual assistant that has access to such information.", function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id=None, id_=None), index=0, finish_reason='stop')] created=1735909199 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=19, completion_tokens=56, total_tokens=75) object_='chat.completion'


* Force calling a function.

In [137]:
# Такая функция в GigaChat в таком виде не работает
# Суть, что функция получения погоды из промпта работает в принудительном 
# порядке, и даже если там ничего нет, то аргументы всё равно будут найдены
# (в лекции были моделью найдены те значения, которые были указаны в описании)

"""
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = get_completition(
                messages, 
                functions, 
                function_call={'name': 'get_current_weather'})

print(response)
"""

'\nmessages = [\n    {\n        "role": "user",\n        "content": "hi!",\n    }\n]\nresponse = get_completition(\n                messages, \n                functions, \n                function_call={\'name\': \'get_current_weather\'})\n\nprint(response)\n'

* Final notes.

In [27]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = get_completition(messages, functions, function_call='auto')
print(response)

choices=[Choices(message=Messages(role='assistant', content='', function_call=FunctionCall(name='get_current_weather', arguments={'location': 'Boston'}), name=None, attachments=None, data_for_context=None, functions_state_id='a0fdc22e-9d6c-4d3b-99f3-ce1e71b713bc', id_=None), index=0, finish_reason='function_call')] created=1735912599 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=113, completion_tokens=29, total_tokens=142) object_='chat.completion'


In [28]:
args = response.choices[0].message.function_call.arguments
observation = get_current_weather(**args)

In [33]:
messages.append(
        Messages(
            role= MessagesRole.ASSISTANT,
            content=json.dumps({'result': observation}),
        )
)

In [34]:
response = response = get_completition(messages, functions, function_call='auto')
print(response)

choices=[Choices(message=Messages(role='assistant', content='\n\nВ Бостоне сейчас **22 градуса Цельсия** и солнечно. Ветер также присутствует.', function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id=None, id_=None), index=0, finish_reason='stop')] created=1735912653 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=66, completion_tokens=25, total_tokens=91) object_='chat.completion'


In [35]:
messages = [
    {
        "role": "user",
        "content": "Какая погода в Boston?",
    }
]
response = get_completition(messages, functions, function_call='auto')

args = response.choices[0].message.function_call.arguments
observation = get_current_weather(**args)

messages.append(
        Messages(
            role= MessagesRole.ASSISTANT,
            content=json.dumps({'result': observation}),
        )
)
messages.append(
        Messages(
            role= MessagesRole.USER,
            content="А в Санкт-Петербурге?",
        )
)

response = get_completition(messages, functions, function_call='auto')

args = response.choices[0].message.function_call.arguments
observation = get_current_weather(**args)
messages.append(
        Messages(
            role= MessagesRole.ASSISTANT,
            content=json.dumps({'result': observation}),
        )
)
response = response = get_completition(messages, functions, function_call='auto')
print(response)

choices=[Choices(message=Messages(role='assistant', content='\n\nТакой ответ указывает на то, что погодные условия в Санкт-Петербурге аналогичны тем, которые были заданы для Бостона: температура около 22 градусов и переменная облачность с ветром.', function_call=None, name=None, attachments=None, data_for_context=None, functions_state_id=None, id_=None), index=0, finish_reason='stop')] created=1735912771 model='GigaChat:1.0.26.20' usage=Usage(prompt_tokens=207, completion_tokens=48, total_tokens=255) object_='chat.completion'


In [37]:
response.choices[0].message.content

'\n\nТакой ответ указывает на то, что погодные условия в Санкт-Петербурге аналогичны тем, которые были заданы для Бостона: температура около 22 градусов и переменная облачность с ветром.'